<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/Ego_Vision_%EC%86%90%EB%8F%99%EC%9E%91_%EC%9D%B8%EC%8B%9D_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/01_vgg16_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Baseline] VGG16을 이용한 손동작 예측

[[Baseline] VGG16을 이용한 손동작 예측](https://dacon.io/competitions/official/235805/codeshare/3345?page=1&dtype=recent)




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/dacon/EgoVision/open.zip'

# 패키지 선언

In [4]:
import pandas as pd
import numpy as np
from glob import glob
from io import BytesIO, StringIO
import zipfile

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# GPU 환경 설정
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [5]:
data_path = 'content'

train_path = data_path + '/train'
test_path = data_path + '/test'

In [ ]:
def get_data(s) :
  id, actor, data, image_id = [], [], [], []
  with zipfile.ZipFile(zip_file_path, 'r') as obj:
    zip_namelist = obj.namelist()    # content(작업 디렉토리)에 압축 해제
    for name in zip_namelist :
      if name.endswith('.png') :
        readed = obj.open(name)
        img = Image.open(readed)
        img = img.resize((224, 224))
        img_arr = np.array(img)
        arr = np.append(arr, img_arr)
      elif name.endswith('json') :  # json
        readed = obj.open(name)
        js = json.load(readed)
        length = len(js['annotations'])
        try :
          action = js['action']
        except :
          action = None
        
        image_id.extend([d['image_id'] for d in js['annotations']])
        data.extend([d['data'] for d in js['annotations']])
        id.extend([js['id']]*length)
        actor.extend([js['actor']]*length)
        action.extend([js['action']]*length)


      elif name.endswith('.csv') :   # csv파일은 2가지
        readed = obj.read(name)
        print(name)
        if name[-len('hand_gesture_pose.csv'):] == 'hand_gesture_pose.csv' :
          hand_gesture = pd.read_csv(BytesIO(readed))
        else :
          sample_submission = pd.read_csv(BytesIO(readed))
  answers = pd.DataFrame({'id' : id,'image_id':image_id, 'actor':actor, 'data':data})
  return hand_gesture, sample_submission, 

In [35]:
arr = np.array([])

with zipfile.ZipFile(zip_file_path, 'r') as obj:
    zip_namelist = obj.namelist()    # content(작업 디렉토리)에 압축 해제
    id, actor, data, image_id = [], [], [], []
    for name in zip_namelist :
      if name.endswith('.png') :
        # readed = obj.open(name)
        # img = Image.open(readed)
        # img = img.resize((224, 224))
        # img_arr = np.array(img)
        # arr = np.append(arr, img_arr)
        pass
      elif name.endswith('json') :  # json
        readed = obj.open(name)
        js = json.load(readed)
        length = len(js['annotations'])
        try :
          action = js['action']
        except :
          action = None
        
        image_id.extend([d['image_id'] for d in js['annotations']])
        data.extend([d['data'] for d in js['annotations']])
        id.extend([js['id']]*length)
        actor.extend([js['actor']]*length)

      elif name.endswith('.csv') :   # csv파일은 2가지
        readed = obj.read(name)
        print(name)
        if name[-len('hand_gesture_pose.csv'):] == 'hand_gesture_pose.csv' :
          hand_gesture = pd.read_csv(BytesIO(readed))
        else :
          sample_submission = pd.read_csv(BytesIO(readed))

hand_gesture_pose.csv
sample_submission.csv


KeyboardInterrupt: ignored

In [14]:
answers = pd.DataFrame({'id' : id,'image_id':image_id, 'actor':actor, 'data':data})
len(id), len(actor), len(data), len(image_id), len(zip_namelist)

(5115, 5115, 5115, 5115, 9662)

In [37]:
js

{'action': [157, '엄지,중지 붙이기'],
 'actor': None,
 'annotations': [{'data': [[1109.1493606567383,
     669.9206471443176,
     -0.15929407440125942],
    [1121.4447784423828, 617.0202541351318, -68.04574728012085],
    [1092.6544189453125, 565.7941389083862, -113.17432880401611],
    [1044.5316696166992, 561.0070395469666, -148.7891721725464],
    [1000.9712219238281, 589.6150517463684, -167.40726470947266],
    [986.6520309448242, 555.9746146202087, -55.94467878341675],
    [925.3554725646973, 528.8032472133636, -81.19319915771484],
    [892.5066947937012, 516.7876052856445, -101.29377365112305],
    [865.4391860961914, 508.99084210395813, -114.9553656578064],
    [970.4821014404297, 592.4842858314514, -54.687663316726685],
    [907.8115653991699, 572.8980231285095, -136.04235649108887],
    [948.0022430419922, 584.2800736427307, -181.6537857055664],
    [983.3870315551758, 594.6288728713989, -182.23660469055176],
    [962.706298828125, 631.991229057312, -60.19437789916992],
    [898.301

In [36]:
print("json keys              : ", js.keys())
print("json action info       : ",js.get('action'))
print("json actor info        : ",js.get('actor'))
print("json annotations keys  : ",js.get('annotations')[0].keys())
print(js.get('annotations')[1].get('data'))

json keys              :  dict_keys(['action', 'actor', 'id', 'annotations'])
json action info       :  [157, '엄지,중지 붙이기']
json actor info        :  None
json annotations keys  :  dict_keys(['image_id', 'data'])
[[1114.3463516235352, 675.5391240119934, -0.16859019175171852], [1125.0972747802734, 623.6944484710693, -69.40220832824707], [1093.6460494995117, 569.7270083427429, -115.03918647766113], [1045.1046752929688, 566.2954759597778, -150.19565105438232], [1002.9677581787109, 600.6170439720154, -171.21017932891846], [991.080322265625, 559.6699476242065, -58.78225564956665], [929.4456481933594, 534.7635769844055, -82.33996152877808], [894.2513465881348, 525.2242684364319, -102.38395929336548], [864.6398735046387, 519.9468398094177, -115.52487373352051], [973.2527160644531, 595.2343010902405, -55.44721484184265], [908.5936546325684, 572.5860071182251, -130.7627534866333], [946.897087097168, 581.4152812957764, -171.7406129837036], [982.1168518066406, 591.6686797142029, -170.3865623474121

# Check Sample Data

In [16]:
sample_submission.head()

,Image_Path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,...,Label_177,Label_186,Label_188,Label_189,Label_190,Label_191,Label_192,Label_193,Label_194,Label_195
0,./test\649,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,./test\650,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,./test\651,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,./test\652,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,./test\653,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
hand_gesture.head()

,pose_id,pose_type,pose_name,gesture_type,hand_type,view_type
0,0,number,숫자 1,my_hand,left,1st
1,1,number,숫자 2,my_hand,left,1st
2,2,number,숫자 3,my_hand,left,1st
3,3,number,숫자 4,my_hand,left,1st
4,4,number,숫자 5,my_hand,left,1st


In [ ]:
print(arr.shape)
plt.imshow(arr[0])
plt.axis('off')
plt.show()

# Baseline Modeling

## Preprocessing

In [18]:
classes = pd.get_dummies(answers[['answers']], columns = ['answers']).to_numpy()

KeyError: ignored

In [ ]:
np.random.shuffle(train_folders) # 일반화 가능성 높이기

In [27]:
X = np.array(images)
print('Train X Shape : ', X.shape)

y = np.array(targets)
print('Train y Shape : ', y.shape)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      random_state=2021, 
                                                      test_size=0.2, 
                                                      stratify = y)

print('X_train shape : ', X_train.shape)
print('X_valid shape : ', X_valid.shape)
print('y_train shape : ', y_train.shape)
print('y_valid shape : ', y_valid.shape)

Train X Shape :  (5888, 224, 224, 3)
Train y Shape :  (5888, 157)
X_train shape :  (4710, 224, 224, 3)
X_valid shape :  (1178, 224, 224, 3)
y_train shape :  (4710, 157)
y_valid shape :  (1178, 157)


In [ ]:
test_images  = []
for test_folder in tqdm(test_folders, total = len(test_folders)) :
    image_paths = sorted(glob(test_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
    query_path  = test_folder.replace(data_path,'')
    test_image = []
    for image_path in image_paths:
        img = image.load_img(image_path, target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        test_image.append(img)
    test_images.append(test_image)

test_images = np.array(test_images)
print(test_images.shape)

 23%|██▎       | 50/217 [00:35<01:52,  1.49it/s]

# Modeling

In [ ]:
baseModel = VGG16(weights='imagenet', include_top=False)
baseModel.trainable = False

model_in = Input(shape = (224,224,3))
base_model = baseModel(model_in)
head_model = AveragePooling2D(pool_size=(7, 7))(base_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.2)(head_model)
model_out = Dense(classes.shape[1], activation="softmax")(head_model)

model = Model(inputs=model_in, outputs=model_out)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.fit(X_train, y_train, 
          validation_data = (X_valid, y_valid),
          epochs=20,
          verbose = 1,
          batch_size=128)

In [ ]:
predictions = []
for test_image in tqdm(test_images, total = len(test_images)) : 
    prediction = np.mean(model.predict(np.array(test_image)), axis = 0)
    predictions.append(prediction)

sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv('./BASELINE.csv', index=False)